## GeoPandas Demo: Get Counties
After the quick introduction, this script explores using GeoPandas in a difference context. Specficially, it looks at the mashup of `GeoPandas` and the `Requests` package to enable us to access features served in an ArcGIS Map Service as a GeoPandas dataframe. 

In [1]:
import requests
import pandas as pd
import geopandas as gpd

%matplotlib inline

### Fetching the data
* Build the request
* Send the request, receive the response

In [2]:
#Build the request and parameters to fetch county features
#  from the NOAA ArcGIS map server end point
stateFIPS = '37' #This is NC

url = 'https://nowcoast.noaa.gov/arcgis/rest/services/nowcoast/mapoverlays_political/MapServer/find'
params = {'searchText':stateFIPS,
          'contains':'true',
          'searchFields':'STATEFP',
          'sr':'',
          'layers':'2',
          'layerDefs':'',
          'returnGeometry':'true',
          'maxAllowableOffset':'',
          'geometryPrecision':'',
          'dynamicLayers':'',
          'returnZ':'false',
          'returnM':'false',
          'gdbVersion':'',
          'returnUnformattedValues':'false',
          'returnFieldName':'false',
          'datumTransformations':'',
          'layerParameterValues':'',
          'mapRangeValues':'',
          'layerRangeValues':'',
          'f':'json'}

In [3]:
#Fetch the data
response = requests.get(url,params)

### Examining the response
* Convert the response to a JSON object
* Examine its structure
* Extract the `attributes` and `geometry` elements.

In [4]:
#Convert to a JSON object (i.e. a dictionary)
respons_js = response.json()

In [5]:
#The 'results' object contains a record for each county returned
results = respons_js['results']
len(results)

100

In [6]:
#Within each item in the results object are the following items
results[0].keys()

dict_keys(['layerId', 'layerName', 'displayFieldName', 'foundFieldName', 'value', 'attributes', 'geometryType', 'geometry'])

In [7]:
#The 'attributes' item contains the feature attributes
results[0]['attributes']

{'FID': '159',
 'Shape': 'Polygon',
 'STATEFP': '37',
 'COUNTYFP': '015',
 'COUNTYNS': '01026334',
 'GEOID': '37015',
 'NAME': 'Bertie',
 'LSAD': '06',
 'ALAND': '1811097673',
 'AWATER': '108727758',
 'AFFGEOID': '0500000US37015'}

In [8]:
#And the geometry object contains the shape
results[0]['geometry']

{'rings': [[[-8608078.87744889, 4310879.947869732],
   [-8607855.681813259, 4311157.199401064],
   [-8607283.054183457, 4311298.513661726],
   [-8607034.255056638, 4311564.619169095],
   [-8606785.344632704, 4312363.801388674],
   [-8606679.591093063, 4312576.761309607],
   [-8606465.968933858, 4312789.725423373],
   [-8604441.846024798, 4313053.388036737],
   [-8604014.71301244, 4313159.460997633],
   [-8603356.146725837, 4313691.63226638],
   [-8603036.771017909, 4313902.138504786],
   [-8602309.965851361, 4314167.756942879],
   [-8601812.256260224, 4314273.703053501],
   [-8601279.258377463, 4314326.194305609],
   [-8599856.483525068, 4314199.857638092],
   [-8599358.662606183, 4314110.444661026],
   [-8599111.978533875, 4313985.6259121755],
   [-8598792.602814078, 4313914.124437068],
   [-8598526.103873806, 4313967.027238429],
   [-8598242.016453078, 4314126.563792094],
   [-8597955.70264633, 4314392.738811895],
   [-8597228.897491638, 4314942.60659493],
   [-8597173.905659784, 431

### Convert the elements to dataFrames
* Creating a dataFrame from the Results object
* "Exploding" the values in the `attributes` and `geometry` columns
* Concatenating dataFrames lengthwise (adding columns)

In [9]:
#Create a dataFrame from the results, 
#  keeping just the attributes and geometry objects
df = pd.DataFrame(results,columns=('attributes','geometry'))
df.head()

,attributes,geometry
0,"{'FID': '159', 'Shape': 'Polygon', 'STATEFP': ...","{'rings': [[[-8608078.87744889, 4310879.947869..."
1,"{'FID': '160', 'Shape': 'Polygon', 'STATEFP': ...","{'rings': [[[-8429869.786865463, 4291798.18384..."
2,"{'FID': '161', 'Shape': 'Polygon', 'STATEFP': ...","{'rings': [[[-8761804.792557526, 4234648.80601..."
3,"{'FID': '162', 'Shape': 'Polygon', 'STATEFP': ...","{'rings': [[[-9160749.25508372, 4257980.584786..."
4,"{'FID': '163', 'Shape': 'Polygon', 'STATEFP': ...","{'rings': [[[-8616429.51120074, 4301301.153104..."


In [10]:
#Explode the dictionary values into fields
dfCounties = df['attributes'].apply(pd.Series)
dfGeom = df['geometry'].apply(pd.Series)

In [11]:
#Combine the two
dfAll = pd.concat((dfCounties,dfGeom),axis='rows')
dfAll.columns

Index(['AFFGEOID', 'ALAND', 'AWATER', 'COUNTYFP', 'COUNTYNS', 'FID', 'GEOID',
       'LSAD', 'NAME', 'STATEFP', 'Shape', 'rings', 'spatialReference'],
      dtype='object')

### Converting the geometry coordinates to a geometric feature
* Exploring the 'rings' object
* Exploring the `shapely` package
* Using shapely to create features
* Converting the dataFrame to geodataFrame
* Plotting the output

In [13]:
#Demo creating a shapely polygnon from the JSON ring
rings = dfAll['rings'][0]
print ("There is/are {} ring(s)".format(len(rings)))
print ("There are {} vertices in the first ring".format(len(rings[0])))
print ("The first vertex is at {}".format(rings[0][0]))

There is/are 2 ring(s)
There are 2 vertices in the first ring
The first vertex is at 0                                                  NaN
0    [[[-8608078.87744889, 4310879.947869732], [-86...
Name: rings, dtype: object


In [14]:
from shapely.geometry import LinearRing
from shapely.geometry import Polygon
ring = rings[0]
r = LinearRing(ring)
s = Polygon(r)

ValueError: A LinearRing must have at least 3 coordinate tuples

In [15]:
#https://shapely.readthedocs.io/en/latest/manual.html#polygons
from shapely.geometry import Polygon
from shapely.geometry import LinearRing
def polyFromRing(ring):
    r = LinearRing(ring)
    s = Polygon(r)
    return r
dfAll['geometry']=dfAll.apply(lambda x: Polygon(x.rings[0]),axis=1)

TypeError: ("'float' object is not subscriptable", 'occurred at index 0')

In [ ]:
gdf=gpd.GeoDataFrame(dfAll)

In [ ]:
gdf[gdf['NAME'] == 'Durham'].plot();

In [ ]:
gdf.to_csv("counties_{}.csv".format(stateFIPS))